In [1]:
# Import Python package
import BinhoPulsar
from BinhoPulsar.Pulsar import Pulsar
from BinhoPulsar.commands.uart.definitions import *


## Getting started

#### 1. List all the Pulsar devices connected to the PC host

The ``BinhoPulsar.getConnectedPulsarDevicesList()`` gets a list of the Pulsar devices plugged into the host PC machine.

In [2]:
BinhoPulsar.getConnectedPulsarDevicesList()

[{'path': '\\\\?\\HID#VID_1FC9&PID_82FD#7&2450afe2&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}',
  'vendor_id': '0x1fc9',
  'product_id': '0x82fd',
  'serial_number': '80F39E5FDBEF2053B16BBC5AB6EFA89',
  'release_number': 256,
  'manufacturer_string': 'Binho LLC',
  'product_string': 'Binho Pulsar',
  'usage_page': 65280,
  'usage': 1,
  'interface_number': 0}]

#### 2. Create an instance of the Pulsar class

To utilize a Pulsar USB host adapter device, we need to create an instance of the Pulsar class.

In [3]:
# Create a device instance. One instance of the Pulsar class represents a Pulsar USB host adapter device.
pulsar = Pulsar()

#### 3. Open connection to the Pulsar device

The public method ``Pulsar.open()`` establishes the connection with a Pulsar device. Below is the complete signature:

```python
open(serial, path)
```

- ``serial: str``: The Pulsar serial number.
- ``path: str``: The OS HID path assigned to the device. This can be obtained using the ``BinhoPulsar.getConnectedPulsarDevicesList()`` method. The ``path`` parameter is currently the only way to uniquely identify each Pulsar device. Therefore, it is recommended to use the ``path`` parameter, especially when opening connections with more than one Pulsar device simultaneously.


In [4]:
# Use the method by default to connect to only one Pulsar device.
pulsar.open()

# Otherwise, use the path attribute to identify each Pulsar device. Uncomment the line below and comment the line #2.
# pulsar.open(path='\\\\?\\HID#VID_1FC9&PID_82FC#6&48d9417&0&0000#{4d1e55b2-f16f-11cf-88cb-001111000030}')

{'module': 0,
 'opcode': 0,
 'message': 'Connection with Pulsar device opened successfully.'}

#### 4. Define and register a callback to handle responses and notifications from Pulsar

To handle responses and notifications from Pulsar, a callback function must be defined and registered. This function will be invoked every time the Pulsar sends a response to a request, an asynchronous notification, or a message from the system.

The callback function's signature is as follows: 

``def callback_function_name(pulsar_message: dict, system_message: dict) -> None:``

Once the callback function is defined, it should be registered using the ``Pulsar.onEvent(callback_function)`` method.

In [5]:
# Define callback function
def callback_function(pulsar_message: dict, system_message: dict) -> None:

    if pulsar_message != None:

        # Print mesasage
        print(">> New message from PULSAR:")
        print(pulsar_message)

    if system_message != None:

        # Print message
        print(">> New message from the SYSTEM:")
        print(system_message)

In [6]:
# Register callback function
pulsar.onEvent(callback_function)

{'module': 0,
 'opcode': 0,
 'message': 'On event callback function registered successfully.'}

#### 5. Define a function to generate transaction IDs

All the request messages sent to the Pulsar from the USB Host application must include the transaction or request ID. The ID is a 2-byte integer with an allowed range of ``[1, 65535]``.

In this example, a dummy function called ``getId()`` is defined to increment a transaction counter used as the ID.

In [8]:
#Auxiliary code to generate IDs.

counter_id = 0

def getId():
    global counter_id
    counter_id = counter_id + 1
    return counter_id

## Using the UART peripheral  

### 1. Connecting TX to RX for UART Loopback Test

To perform the UART test:

**Connection Setup:**
   - Physically connect the TX (transmit) pin to the RX (receive) pin on your device.
   - This direct connection forms a loopback, allowing transmitted data to be immediately received back by the device.

**Expected Messages and Notifications:**
   - Upon writing data:
     - Expect a confirmation of the write status.
     - If the write is successful, anticipate a notification containing the transmitted data.
   
**Testing Process:**
   - Write data using the Pulsars's functionalities.
   - Look for immediate feedback:
     - Confirmation message indicating the write status.
     - Notification displaying the transmitted data if the write operation succeeds.



### 2. Initializing the UART Peripheral

To initialize the UART peripheral:

**Setting UART Peripheral Voltage:**
   - Before initializing, ensure the UART peripheral voltage is appropriately configured (3.3V in this case).
   - Proper voltage settings are crucial for the correct operation of the UART communication.

``Pulsar.setI2cSpiUartGpioVoltage(id, voltage_mV)``

This method supplies the indicated voltage to the bus in mV, ranging from 1200 mV up to 3300 mV. It also allows setting the value 0 mV to power off the output voltage.

- ``id: int`` : A 2-byte integer that represents the transfer ID.
- ``voltage_mV: c_int16 ``: The voltage parameter is a 2-byte integer in the range [1200, 3300] mV and allows the value 0 mV to power off the output voltage.

In [9]:
pulsar.setI2cSpiUartGpioVoltage(getId(),3300)

{'module': 1,
 'opcode': 0,
 'message': 'SET I2C SPI UART GPIO VOLTAGE request success'}

>> New message from PULSAR:
{'id': 1, 'command': 'SYS SET I2C SPI UART GPIO VOLTAGE', 'result': 'SUCCESS'}


**Initialize UART:**

When initializing the UART peripheral, consider the following configurable parameters:

* _Baudrate:_  
   - Defines the speed of data transmission, ranging from 600 bps to 115200 bps.  

* _Hardware Handshake:_
   - Enables or disables the Request to Send/Clear to Send (RTS/CTS) signal for flow control.  

* _Parity Mode:_ 
   - Refers to the parity check used in UART packages, offering error detection.

* _Data Character Size:_ 
   - Determines the length of each UART data package in bits.
   
* _Stop Bit:_  
   - Specifies the number of stop bits used at the end of each UART transaction.

For this case we are using:

* Baudrate: 600bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data size
* Stop Bit: single stop bit (1 bit stop)  



In [10]:
pulsar.uartInit(getId(),UartBaudRate.UART_BAUD_600, False, UartParity.UART_NO_PARITY, UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART INIT request success'}

>> New message from PULSAR:
{'id': 2, 'command': 'UART INIT', 'result': 'SUCCESS'}


### 3. Sending Data via UART

The Pulsar system can transmit data via UART, the data to be transmitted must adhere to specific criteria:

**Data Format:**
   - The data format consists of an array with a length less than 1024.
   - Each element in the array represents a byte value, constrained within the range of 0x00 to 0xFF.

For this test an array with 1024 elements, where each value stays within the byte range by using the modulo operation (`%`) with `0xFF` is used.


In [11]:
data = [i%0xFF for i in range(0,70)]

In [12]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 3, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


## Change the UART peripheral parameters

Once the UART peripheral is initialized, adjustments to its configuration can be made using the _uartSetParameters_ function. This function allows modification of the following parameters, similar to the initialization:

* _Baudrate:_  
   - Defines the speed of data transmission, ranging from 600 bps to 115200 bps.  

* _Hardware Handshake:_
   - Enables or disables the Request to Send/Clear to Send (RTS/CTS) signal for flow control.  

* _Parity Mode:_ 
   - Refers to the parity check used in UART packages, offering error detection.

* _Data Character Size:_ 
   - Determines the length of each UART data package in bits.
   
* _Stop Bit:_  
   - Specifies the number of stop bits used at the end of each UART transaction.


### Baudrate 1200 bps

For this case we are using:

* Baudrate: 1200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 


In [13]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_1200,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 4, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [14]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 5, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 2400 bps

For this case we are using:

* Baudrate: 2400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [15]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_2400,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 6, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [16]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 7, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 4800 bps

For this case we are using:

* Baudrate: 4800 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [17]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_4800,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 8, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [18]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 9, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 9600 bps

For this case we are using:

* Baudrate: 9600 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [19]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_9600,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 10, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [20]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 11, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 14400 bps

For this case we are using:

* Baudrate: 14400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [21]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_14400,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 12, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [22]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 13, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 19200 bps

For this case we are using:

* Baudrate: 19200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [23]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_19200,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 14, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [24]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 15, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 38400 bps

For this case we are using:

* Baudrate: 38400 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [25]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_38400,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 16, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [26]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 17, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 56000 bps

For this case we are using:

* Baudrate: 56000 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [27]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_56000,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 18, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [28]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 19, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 57600 bps

For this case we are using:

* Baudrate: 57600 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [29]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_57600,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 20, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [30]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 21, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 115200 bps

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: No parity
* Data size: 8 bit data
* Stop Bit: 1 bit stop 

In [31]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_115200,False,UartParity.UART_NO_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 22, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [32]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 23, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 115200 bps and Even Parity

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 8 bit data 
* Stop Bit: 1 bit stop 

In [33]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_115200,False,UartParity.UART_EVEN_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 24, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [34]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 25, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 115200 bps, Even Parity and 2 bit stop

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 8 bit data 
* Stop Bit: 2 bit stop 

In [35]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_115200,False,UartParity.UART_EVEN_PARITY,UartDataSize.UART_8BIT_BYTE,UartStopBit.UART_TWO_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 26, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [36]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 27, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


### Baudrate 115200 bps, Even Parity, 7 bit data character and 2 stop bit

For this case we are using:

* Baudrate: 115200 bps
* Hardware handshake: False
* Parity Mode: Even
* Data size: 7 bit data 
* Stop Bit: 2 bit stop 

In [37]:
pulsar.uartSetParameters(getId(),UartBaudRate.UART_BAUD_115200,False,UartParity.UART_EVEN_PARITY,UartDataSize.UART_7BIT_BYTE,UartStopBit.UART_ONE_STOP_BIT)

{'module': 1, 'opcode': 0, 'message': 'UART SET PARAMETERS request success'}

>> New message from PULSAR:
{'id': 28, 'command': 'UART SET PARAMETERS', 'result': 'SUCCESS'}


In [38]:
pulsar.uartSendMessage(getId(),data)

{'module': 1, 'opcode': 0, 'message': 'UART SEND MESSAGE request success'}

>> New message from PULSAR:
{'id': 29, 'command': 'UART SEND', 'result': 'SUCCESS'}
>> New message from PULSAR:
{'id': 0, 'command': 'UART RECEIVE NOTIFICATION', 'result': 'SUCCESS', 'payload_length': 70, 'payload': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]}


## Close communication

Use the ``Pulsar.close()`` method to end the communication with the Pulsar device and release the used memory in the background like threads and so on.

In [39]:
# Close the communication with the Pulsar device.
pulsar.close()

{'module': 0, 'opcode': 0, 'message': 'Communication closed successfully.'}